# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [27]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


/home/ammar594/.local/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['exp', 'clf', 'permutation']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [28]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['Stand', 'Frog', 'Right', 'Belly', 'HeadBack', 'Crouch', 'Knee', 'Left', 'Back', 'StandInit', 'Sit']


In [29]:
class StrToBytes:
    def __init__(self, fileobj):
        self.fileobj = fileobj
    def read(self, size):
        return self.fileobj.read(size).encode()
    def readline(self, size=-1):
        return self.fileobj.readline(size).encode()

In [30]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    data_file = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data_temp = StrToBytes(open(data_file))
    data = pickle.load(data_temp)
    target = [i] * len(data)
    return data, target

In [31]:
# load all the data
all_data = []
all_target = []

for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)

print ('total number of data', len(all_data))

total number of data 222


In [32]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [33]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [34]:
train_dataset = []
train_targetset = []
for index in permutation:
    train_dataset.append(all_data[index])
    train_targetset.append(all_target[index])
    

clf.fit(train_dataset[:n_training_data], train_targetset[:n_training_data])

SVC(C=100.0, gamma=0.001)

In [35]:
dataset = []
targetset = []
for i in permutation:
    dataset.append(all_data[i])
    targetset.append(all_target[i])
    
print (permutation)
clf.fit(all_data[:n_training_data], all_target[:n_training_data])

[151 197 183  15  57 212  18  81 174 213  20 101 166 157 155  90  79 112
  93 159  75  25  41 173 158 121 136 164 208 146  30 149  94  66  69  13
 200  70 102 188  72 144  89 126   7 113  86 140  42  27 108 218  21  14
  31  84  23 177 198 203 163  60 205 182  37  43   6  48  24  88  49 204
 103  67  56 117 176 105  39 195 109 152 190 187 206 142 119  92 180   0
 215  95 160 106 207 107 191 220  32 147  22  52 219   4  73 115 114 132
  34  40  68 125 130  58  54 129  98  63 175 118  59 138 194  99  51  36
 193 192 122  46 111 120  26 178 181 156  16 135  47 217  17  74  87  12
 171 172 161 110 153  50  38   2 148  80  61 209 133 165 145 104 184  83
 221   1  19   3  76 154 128  35 143  82 210 169  29 211  45 201 179 137
  65  85  78   9 100  71 199 123 162  53 150 189 202 124  10 170  55  28
 131  91  77  11 216 186 116 167   5  97 214 139 185   8 141  44 134  33
  64 127 196  62  96 168]


SVC(C=100.0, gamma=0.001)

### predicting

In [36]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([9]), [10])

In [37]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [38]:
expected = []
predicted = []
# YOUR CODE HERE
predicted = clf.predict(all_data[:n_training_data])
expected = all_target[:n_training_data]
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        10
           2       1.00      0.91      0.95        11
           3       1.00      1.00      1.00        19
           4       1.00      1.00      1.00        10
           5       0.97      1.00      0.98        30
           6       1.00      1.00      1.00        10
           7       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        23
           9       1.00      1.00      1.00        11

    accuracy                           0.99       155
   macro avg       1.00      0.99      0.99       155
weighted avg       0.99      0.99      0.99       155


Confusion matrix:
[[11  0  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0  0]
 [ 0  0 10  0  0  1  0  0  0  0]
 [ 0  0  0 19  0  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0 30  0  0  0  0]
 

## 4. Evaluate on the test data

In [39]:
expected = []
predicted = []
# YOUR CODE HERE
testData = permutation[n_training_data:]
testData.shape
expected = np.asarray(all_target)[testData]
predicted = clf.predict(np.asarray(all_data)[testData])
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         2
           5       0.91      1.00      0.95        10
           6       1.00      1.00      1.00         4
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00        11
           9       0.61      1.00      0.76        14
          10       0.00      0.00      0.00        10

    accuracy                           0.85        67
   macro avg       0.87      0.91      0.88        67
weighted avg       0.76      0.85      0.79        67


Confusion matrix:
[[ 6  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  0  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  0  0  0  0  0  0]
 

/home/ammar594/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ammar594/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ammar594/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [40]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [41]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data[-1:]), all_target[-1:]

(array([9]), [10])

In [42]:
all_data[-1:]

[[-0.3128941059112549,
  -0.230057954788208,
  -1.6704840660095215,
  0.9663779735565186,
  -0.3128941059112549,
  0.0353238582611084,
  -1.6228026151657104,
  0.8299360275268555,
  0.025310691446065903,
  -0.16758745908737183]]